In [1]:
import json
import requests
import re
#now query the edgematch endpoint to get the list of service
url = 'https://gistmaps.itos.uga.edu/arcgis/rest/services/COD_External_Edgematch?f=json'
ret = requests.get(url)
      
if ret.status_code > 206:
    raise Exception(' application did not handle import success message properly.')
obj = json.loads(json.dumps(ret.json()))
svcLUs = []
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
layers = map.listLayers()
for i in obj["services"]:
    if i['type'] == 'MapServer':
        if not re.search('pcode', i['name']):
            url2 = 'https://gistmaps.itos.uga.edu/arcgis/rest/services/' + i["name"] +'/MapServer/0'
            #print(url2)
            map.addDataFromPath(url2)
#All the names will be admin 0 in the table of contents lets make those more informative of the location
for l in map.listLayers():
    w = l.dataSource
    p = w[74:-12]
    if re.search('Edgematch', w):
        l.name = p
        sym = l.symbology
        sym.renderer.symbol.applySymbolFromGallery("Extent Transparent Wide Gray")
        sym.renderer.symbol.color = {'RGB': [255, 0, 0, 60]}
        sym.renderer.symbol.outlineColor = {'CMYK' : [25, 50, 75, 25, 100]}
        sym.renderer.symbol.size = 2

        l.symbology = sym
    
    
